<a href="https://colab.research.google.com/github/ItzWarLockOp/Google-Colab-Scriptz/blob/main/Link2Onedrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><img src="https://i.ibb.co/f9Swky3/IMG-20230528-165007.png" height="150" alt="Rclone-Logo"/></center>

#<h4> <b>[Step : 1️⃣]

#<h3><b><font color='orange'>📲Install Rclone & Setup Libraries 📚

In [ ]:
#@title <b>⬅️Run this cell to Install Rclone📲

from sys import exit as end
from pathlib import Path
from requests import get as get_url
from google.colab import output
from json import dumps as dump_json

current_version = "v5.1.0"
do_normal_setup = "1"

# Check for latest version
try:
    print("Connecting to CDN...")
    json_data = get_url("https://raw.githubusercontent.com/ItzWarLockOp/Trash/main/mct.json").json()
    output.clear()
    print("Connected with CDN.")
    if current_version == json_data["version"]:
        print("Checking current version...")
        print(f'Great Job! You are using the latest version: {json_data["version"]}')
    else:
        print("Checking current version...")
        print(f'Oh-No! You are not using the latest version and missing out on important fixes & features. The latest version is {json_data["version"]}. Please use the latest notebook available here:\n{json_data["notebookLink"]}')
except Exception as error_code:
    end(f'Failed to connect with CDN!\nTraceback:\n{error_code}')

# Save json data locally
with open('mct.json', 'w+') as mct_json:
    mct_json.write(dump_json(json_data))


# User Input
INSTALL_RCLONE_FLAVOR = ""

# Install rclone
try:
    print("Installing rclone...")
    !curl -s https://rclone.org/install.sh | sudo bash {INSTALL_RCLONE_FLAVOR} > /dev/null 2>&1
    output.clear()
    print("Rclone installation completed!")
except Exception as error_code:
    print(f'Failed to install rclone!\nTraceback:\n{error_code}')

if do_normal_setup == '1' and not Path("/content/setup.sh").exists():
    try:
        print("Setting up Libraries...")
        get_url(json_data["setupFile"])
        !curl -s {json_data["setupFile"]} | sudo bash {INSTALL_RCLONE_FLAVOR} > /dev/null 2>&1
        !curl {json_data["setupFile"]} > "/content/setup.sh"
        output.clear()
        print("Setup completed!")
    except ConnectionError:
        print('Unable to get setup file!')
    except Exception as error_code:
        print(f'Setup instructions seem to have mistakes!\nTraceback:\n{error_code}')
elif do_normal_setup == '1' and Path("/content/setup.sh").exists():
    output.clear()
    print("Setup file exists and is already configured!")

#<h4> <b>[Step : 2️⃣]

#<center><img src="https://cdn.thecaduceus.eu.org/MCT/img/rclone_logo.png" height="50" alt="Rclone-Logo"/></center><center><h3> <font color='lime'><b> Upload [Rclone.Conf] File</b></h3></center>

In [ ]:
#@markdown <b>  ⬅️ Run this Cell to Initiate Upload 📲
from google.colab import files
from sys import exit as end
from pathlib import Path, PurePath
from IPython.display import clear_output

UPLOAD_CONFIG_FILE = True

if UPLOAD_CONFIG_FILE:
    # Initiate Upload Request
    print('Request: Initiate upload -> rclone.conf [file]')
    uploaded_files = files.upload()
    if len(uploaded_files) == 0:
        end('File Error: No file uploaded')
    
    uploaded_file_name = next(iter(uploaded_files))
    if uploaded_file_name != 'rclone.conf':
        end('File Error: File is not rclone.conf')

    file_content = uploaded_files[uploaded_file_name]
    rclone_conf_path = Path("/root/.config/rclone/")
    rclone_conf_path.mkdir(parents=True, exist_ok=True)
    rclone_conf_file_path = rclone_conf_path / "rclone.conf"
    with open(rclone_conf_file_path, 'wb') as f:
        f.write(file_content)
    
    print('Rclone.conf uploaded successfully')

# Clear output and show last print message
clear_output(wait=True)
print('Rclone config File uploaded!')

#<h4> <b>[Step : 3️⃣]

#<center> <h3> <font color='sky blue'> <b>Mount & Unmount Onedrive 📁

In [ ]:

from sys import exit as end
from os import makedirs, system
from pathlib import Path, PurePath
from re import findall, M

if not Path("/root/.config/rclone/rclone.conf").exists():
    end('Not Found: rclone.conf [file]')

content = open("/root/.config/rclone/rclone.conf").read()
avCon = findall(r"^\[(.+)\]$", content, M)

print("Available Rclone Remotes:", end=" ")
print(*avCon, sep=", ")

if len(avCon) == 0:
    end('No remotes found in rclone.conf [file]')

remote = avCon[0]

Mount_Point = f"/content/{remote}"
Cache_dir = "/content/temp/Rclone"

Action = "mount" #@param ["mount", "unmount"] {type:"string"}

if Action not in ["mount", "unmount"]:
    end("Input Error: Value should be 'mount' or 'unmount'.")
elif Action == "mount":
    makedirs(Mount_Point, exist_ok=True)
    !rclone mount {remote}: {Mount_Point} --user-agent 'Mozilla' --buffer-size 256M --transfers 10 --vfs-cache-mode minimal --vfs-read-chunk-size 500M --vfs-cache-max-size 50G --vfs-cache-max-age 0h0m1s --vfs-cache-poll-interval 0m1s --cache-dir {Cache_dir} --allow-other --daemon
    print(f'{remote} mounted: {Mount_Point}') if Path(Mount_Point).exists() else print(f"Mount Error: Unable to mount {remote} at {Mount_Point}")
elif Action == "unmount":
    if system(f"fusermount -uz {Mount_Point}") == 0:
        print(f'{remote} unmounted')
    else:
        print(f"Unable to unmount {remote}")
else:
    system(f"fusermount -uz {Mount_Point}")

#<h4> <b>[Step : 4️⃣]

#<center> </font> <h3><b> <font color='yellow'> 🖥️Enter Url, Path & New Name📡  </center>

In [ ]:
#@title <center> <b><h5> • If you want to change your file name then set the [RenameFile] to True✅ • </b></h5></center>
#@markdown ------

import os
import requests
from tqdm import tqdm
import urllib.parse
import math

os.environ['COLAB_SKIP_MAGIC'] = '1'

# Get user input for the download link and path
DownloadLink = "https://DirectLink.com/File.jpeg" #@param {type:"string"}
Path = "/content/drive/My Drive" #@param {type:"string"}

# Extract the filename from the download link
filename = DownloadLink.split("/")[-1]

# Decode the filename
decoded_filename = urllib.parse.unquote(filename)

# Find the position of the last dot in the filename
last_dot_index = decoded_filename.rfind(".")

# Extract the name and extension
name = decoded_filename[:last_dot_index]
extension = decoded_filename[last_dot_index:]

# Get user input for the desired name if renaming is selected
RenameFile = False #@param {type:"boolean"}

if RenameFile:
    NewName = "x" #@param {type:"string"}
else:
    # Remove URL encoding from the name
    NewName = urllib.parse.unquote(name)

#@markdown ------

# Generate the new filename with the original extension
new_filename = f"{NewName}{extension}"

# Download the file and save it to the specified path with the new filename
response = requests.get(DownloadLink, stream=True)
total_size = int(response.headers.get("content-length", 0))

# Convert the total size to a human-readable format
size_units = ["B", "KB", "MB", "GB", "TB"]
size_index = int(math.floor(math.log(total_size, 1024)))
total_size_hr = f"{round(total_size / (1024 ** size_index), 2)} {size_units[size_index]}"

# Print the file name, total size, and save path
print("File Name:", new_filename)
print("File Size:", total_size_hr)
print("Save Path:", os.path.join(Path, new_filename))
print()

# Download the file with progress bar
with open(f"{Path}/{new_filename}", "wb") as file:
    progress_bar = tqdm(total=total_size, unit="B", unit_scale=True, ncols=80, bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt}')
    for data in response.iter_content(chunk_size=1024):
        file.write(data)
        progress_bar.update(len(data))
    progress_bar.close()

# Add an extra line space
print("\n")

# Print completion message
print("📲 File Downloaded Successfully! 🎉")